# Setup

In [1]:
%load_ext autoreload
%autoreload 2

# module
import numpy as np
import os

# read vocab
from helpers import read_vocab
import operator

# Build and read vocabulary with occurrences
Use the full tweet collection

In [2]:
# change path
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_epfl/full/')

In [ ]:
# build vocab pos
# adapt the names the build vocab shell function 
os.system('build_vocab.sh')
os.system('cut_vocab.sh')

In [ ]:
#  build vocab neg
# adapt the names the build vocab shell function 
os.system('build_vocab.sh')
os.system('cut_vocab.sh')

In [3]:
# read the vocabs
vocab_neg = read_vocab('vocab_pos_full.txt')
vocab_pos = read_vocab('vocab_neg_full.txt')

# Extract the most frequent words 

In [4]:
def extract_most_freq(vocab_dict, max_words=10):
    
    most_freq_words = []
    vocab_list_sorted = sorted(vocab_dict.copy().items(), key=operator.itemgetter(1), reverse = True)

    for i in range(max_words):
        most_freq_words.append((vocab_list_sorted[i][0], vocab_list_sorted[i][1]))
    
    return most_freq_words

### in the positive vocabulary 

In [5]:
most_neg = extract_most_freq(vocab_neg, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_neg))

﻿<user>	951004
!	585557
i	543778
.	499484
you	386420
to	317352
,	305792
the	297102
a	239967
is	239662


### in the negative vocabulary 

In [6]:
most_pos = extract_most_freq(vocab_pos, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_pos))

.	751865
i	625066
<user>	557268
(	461668
<number>	420633
<repeat>	414073
the	384681
<url>	359419
,	357807
!	356189


# Compute the occurence ratios for all the words

In [7]:
def word_ratio_neg_pos(vocab_dict_neg, vocab_dict_pos):
    
    word_ratio = {}

    # scan negative tweets
    for word_neg, occu_neg in vocab_dict_neg.items():  
        
        # if word in both neg and pos vocab
        if word_neg in vocab_dict_pos:
            
            # get occurence in positive tweets
            occu_pos = vocab_dict_pos.get(word_neg)
            
            # compute the ratio
            word_ratio[word_neg] = [occu_neg/occu_pos, occu_neg, occu_pos]
            
    return word_ratio

In [8]:
# frequence threshold
freq_min = 100

# threshold the vocabularies with the defined threshold
vocab_neg_thr = {k: v for k, v in vocab_neg.copy().items() if v > freq_min}
vocab_pos_thr = {k: v for k, v in vocab_pos.copy().items() if v > freq_min}

In [9]:
# compute the word ratios
word_ratio = word_ratio_neg_pos(vocab_neg_thr, vocab_pos_thr)

In [10]:
# some example
print(word_ratio.get('happy'))
print(word_ratio.get('not'))

[4.415038919077691, 30062, 6809]
[0.6593738685461874, 136586, 207145]


# Build exclusion vocab with stop words

In [11]:
# parameters
ratio_margin = 0.1
freq_min = 100

# build stop word dictionnary
stop_vocab = {}
for word, values in word_ratio.items():
    
    ratio = values[0]
    occu_neg = values[1]
    occu_pos = values[2]
    
    # remove non discriminant word
    if ratio > 1 - ratio_margin and ratio < 1 + ratio_margin:
        stop_vocab[word] = values
        
    # remove unfrequent word
    if occu_neg < freq_min and occu_pos < freq_min:
        stop_vocab[word] = values
    
print("Number of stop words: ", len(stop_vocab))

Number of stop words:  634


In [12]:
print("\n".join("{}\t{}".format(k,v) for k,v in stop_vocab.items()))

'	[0.9608963231813161, 30446, 31685]
*	[1.0892476266153652, 26045, 23911]
\	[1.0154130702836004, 1647, 1622]
_	[1.0092165898617511, 5256, 5208]
__	[1.0381944444444444, 897, 864]
{	[0.9893992932862191, 560, 566]
<number>d	[0.9299014238773274, 2547, 2739]
<number>years	[1.0165289256198347, 123, 121]
<number>yrs	[0.9230769230769231, 120, 130]
=	[1.0392685851318946, 3467, 3336]
->	[1.0397727272727273, 366, 352]
a	[1.0375650188733088, 239967, 231279]
aah	[1.0, 191, 191]
abby	[1.0980392156862746, 112, 102]
abt	[1.0481540930979134, 653, 623]
achieve	[0.9315068493150684, 136, 146]
act	[1.0745573159366262, 1153, 1073]
actor	[1.0886075949367089, 258, 237]
actually	[1.0196907817969663, 6991, 6856]
adam	[1.0168067226890756, 484, 476]
add	[0.9809230769230769, 1594, 1625]
address	[1.045822102425876, 388, 371]
after	[0.974070351758794, 9692, 9950]
ah	[1.0756044749188018, 5961, 5542]
alex	[0.9923312883435583, 647, 652]
almost	[0.935656836461126, 3141, 3357]
already	[0.9264408493427705, 7330, 7912]
alt

In [14]:
# adapt path 
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [ ]:
# write stop word in a text
outputfilename = "stop_word_" + "freq_min_" + str(freq_min) + "_ratio_marg_" + str(ratio_margin) + ".txt"
with open(outputfilename, 'w', encoding='utf-8-sig') as outfile:
    for word, value in stop_vocab.items():
        outfile.write(word)   
        outfile.write("\n")      
outfile.close()